---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

# Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np
address = pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')
train = pd.read_csv('train.csv',encoding = "ISO-8859-1")
train = train[(train['compliance'] == 0) | (train['compliance'] == 1)]
df_train = pd.merge(train, address,how = 'left', on = 'ticket_id')
df_train = pd.merge(df_train, latlons,how = 'left', on = 'address')
df_train = df_train.set_index('ticket_id')
df_train = df_train.fillna(method = 'pad')
test = pd.read_csv('test.csv')
df_test = pd.merge(test, address,how = 'left', on = 'ticket_id')
df_test = pd.merge(df_test, latlons,how = 'left', on = 'address')
df_test = df_test.set_index('ticket_id')
df_test = df_test.fillna(method = 'pad')
y_train = df_train['compliance']


/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
features = ['disposition','fine_amount','admin_fee','state_fee','late_fee','discount_amount','clean_up_cost', \
            'judgment_amount', 'lat', 'lon']
X_train = df_train[features].fillna(method = 'pad')
X_test = df_test[features].fillna(method = 'pad')
X_train = pd.get_dummies(X_train,columns= ['disposition'])
X_test = pd.get_dummies(X_test,columns = ['disposition'])
train_col = list(X_train.columns)
test_col = list(X_test.columns)
for col in train_col:
    if col not in test_col:
        train_col.remove(col)
X_train = X_train[train_col]
X_test = X_test[train_col]


In [72]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.fit_transform(X_test)


(61001, 13)

In [76]:
#from sklearn.svm import SVC
#from sklearn.metrics import roc_curve, auc
#svc = SVC().fit(X_train_scale,y_train)
#y_score = svc.decision_function(X_train_scale)
#fpr_svc, tpr_svc,thresholds_svc = roc_curve(y_train, y_score)
#roc_auc_svc = auc(fpr_svc, tpr_svc)
#roc_auc_svc

0.50730901786198135

In [51]:
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.metrics import roc_curve, auc
#dt = DecisionTreeClassifier().fit(X_train,y_train)
#y_prob_tree = dt.predict_proba(X_train)
#y_prob_tree = [x[1] for x in y_prob_tree]
#fpr_tree, tpr_tree, thresholds_tree = roc_curve(y_train, y_prob_tree)
#roc_auc_tree = auc(fpr_tree, tpr_tree)
#roc_auc_tree

0.99910624966196737

In [75]:
def blight_model():
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neural_network import MLPClassifier
    clf = MLPClassifier(hidden_layer_sizes =100, alpha = 5, random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scale, y_train)
    y_prob_tree = clf.predict_proba(X_test_scale)
    X_test['compliance'] = y_prob_tree[:,1]
    return X_test['compliance']
blight_model()

ticket_id
284932    0.001283
285362    0.000486
285361    0.001112
285338    0.009835
285346    0.013184
285345    0.010074
285347    0.015077
285342    0.238169
285530    0.009228
284989    0.002915
285344    0.013405
285343    0.003003
285340    0.003388
285341    0.015079
285349    0.013291
285348    0.010156
284991    0.002914
285532    0.015995
285406    0.000986
285001    0.004287
285006    0.002408
285405    0.000482
285337    0.000570
285496    0.002689
285497    0.002062
285378    0.000418
285589    0.000729
285585    0.001211
285501    0.002646
285581    0.000406
            ...   
376367    0.000766
376366    0.001299
376362    0.006764
376363    0.007922
376365    0.000766
376364    0.001299
376228    0.004294
376265    0.001468
376286    0.020087
376320    0.003034
376314    0.002446
376327    0.102448
376385    0.041499
376435    0.293212
376370    0.172678
376434    0.001009
376459    0.004044
376478    0.000208
376473    0.002637
376484    0.003981
376482    0.000561
37